# ヤングの二重スリット実験

In [ ]:
import numpy as np
from math import cos,pi
from matplotlib import pyplot,animation,rc
from IPython.display import HTML

In [ ]:
s = 1.0     # 波の伝わる速さ
L = 2.0     # 系の長さ
T = 4.0     # シミュレーション時間
n = 200     # 空間刻み数
m = 800     # 時間刻み数
dx = L/n    # 空間刻み幅
dt = T/m    # 時間刻み幅
a = s*dt/dx # α

In [ ]:
# 初期条件
x, y = np.meshgrid(np.linspace(0,L,n,False),np.linspace(0,L,n,False),indexing='ij')
u0 = np.zeros((n,n)) # 初期変位
v0 = np.zeros((n,n)) # 初期速度

In [ ]:
# 壁 (穴の空いているところは1)
iw = 100 # 壁のx方向の位置
wall = np.zeros(n)
wall[87:90] = 1   # 穴の空いているところは1
wall[110:113] = 1

In [ ]:
# 強制振動
w = 2*pi*10 # 各振動数
def oscillate(k):
    return 0.2*cos(w*dt*k)# 強制振動

In [ ]:
# シミュレーション
u = np.zeros((m,n,n))
u[0,:,:] = u0[:,:]
u[0,0,:] = oscillate(0)
u[0,iw,:] = u[0,iw,:] * wall[:]
u[1,1:-1,1:-1] = (1-2*a**2)*u[0,1:-1,1:-1]+dt*v0[1:-1,1:-1] \
  +(a**2/2)*(u[0,0:-2,1:-1]+u[0,2:n,1:-1]+u[0,1:-1,0:-2]+u[0,1:-1,2:n])
u[1,0,:] = oscillate(1)
u[1,iw,:] = u[1,iw,:] * wall[:]
for k in range(2,m):
    u[k,1:-1,1:-1] = 2*(1-2*a**2)*u[k-1,1:-1,1:-1]-u[k-2,1:-1,1:-1] \
      +a**2*(u[k-1,0:-2,1:-1]+u[k-1,2:n,1:-1]+u[k-1,1:-1,0:-2]+u[k-1,1:-1,2:n])
    u[k,0,:] = oscillate(k)
    u[k,iw,:] = u[k,iw,:] * wall[:]

In [ ]:
# 等高線プロット
xi = 95  # x = 0.95-1.5
xf = 150
yi = 50  # y = 0.5-1.5
yf = 150
ti = 200 # t = 1.0-1.6
tf = 320
def update(k):
    pyplot.cla()
    pyplot.contourf(x[xi:xf,yi:yf], y[xi:xf,yi:yf], u[k+ti,xi:xf,yi:yf])
    pyplot.title('step='+str(k+ti))
    pyplot.gca().set_aspect('equal');
fig = pyplot.figure()
movie = animation.FuncAnimation(fig,update,frames=tf-ti,interval=50)
rc('animation', html='jshtml')
movie

In [ ]:
# 等高線プロット (t = 305)
t = 305
fig = pyplot.figure()
pyplot.contourf(x[xi:xf,yi:yf], y[xi:xf,yi:yf], u[t,xi:xf,yi:yf])
pyplot.title('step='+str(t))
pyplot.gca().set_aspect('equal');
# pyplot.savefig('young-1.pdf') # ファイルへの出力

In [ ]:
# x=1.1の断面
xp = 110 # x = 1.1
yi = 80  # y = 0.8-1.2
yf = 120
ti = 200 # t = 1.0-2.0
tf = 400
def update(k):
    pyplot.cla()
    pyplot.plot(y[xp,yi:yf], u[k+ti,xp,yi:yf],"b")
    pyplot.ylim(-0.1,0.1)
    pyplot.title('step='+str(k+ti))
fig = pyplot.figure();
movie = animation.FuncAnimation(fig,update,frames=tf-ti,interval=50)
rc('animation', html='jshtml');
movie

In [ ]:
# x=1.1の断面 (t=310)
t = 310
fig = pyplot.figure();
pyplot.plot(y[xp,yi:yf], u[t,xp,yi:yf],"b")
pyplot.ylim(-0.1,0.1)
pyplot.title('step='+str(t))
# pyplot.savefig('young-2.pdf') # ファイルへの出力